In [111]:
import numpy as np
import os

Hacky code to pull out just the local acceleration data across joints / frames

In [ ]:
def convert_file(file_in, file_out):
    file = open(file_in, "r")
    full_data = file.readlines()
    i = full_data.index("MOTION\n")
    frames = int(full_data[i+1].strip().replace("Frames: ", ""))
    frame_time = float(full_data[i+2].strip().replace("Frame Time: ", ""))
    # print(frames, frame_time)

    pos_data = [] # frame [ 83 joints [xpos, ypos, zpos, zrot, xrot, yrot]]]
    for frame in full_data[i+3: ]:
        pos_data.append( [float(x) for x in frame.split()])
    
    # take the derivative to get acceleration
    pos_data = np.array(pos_data)
    v_data = np.diff(pos_data, 1, 0)
    acc_data = np.diff(pos_data, 2, 0)
    
    # How much does taking the derivative reduce the amount of non zero data?
    total_size = np.size(pos_data)
    print(total_size , round(np.count_nonzero(pos_data)/total_size*100,4), round(np.count_nonzero(acc_data)/total_size*100, 4))
    # ~ 70 -> 18%
    
    # cut out the first ~5 seconds on each side of the take to remove weird artifacts
    acc_data = acc_data[500: -500]
    
    np.savetxt(file_out, acc_data, delimiter=',', fmt='%d')

    file.close()
    return frame_time, len(acc_data)

Go through each pair of bvh files and convert to csv with new naming convention

In [146]:
data_dir = "mocap_data"

frames = []
frame_length = []

# iterate through the session directories
for session_dir in os.listdir(data_dir):
    if not session_dir.startswith("session"):
        continue
    session_path = os.path.join(data_dir, session_dir)
    for take_dir in os.listdir(session_path):
        if not take_dir.startswith("take"):
            continue
        take_path = os.path.join(session_path, take_dir)
        file_a, file_b =  [x for x in os.listdir(take_path) if ".bvh" in x]
        person_a = file_a.replace(take_dir + "_hasFingers_", "").replace("_scale_local.bvh", "")
        person_b = file_b.replace(take_dir + "_hasFingers_", "").replace("_scale_local.bvh", "")
        new_a = "csv_data/" + person_a + "_" + person_b + "_" + session_dir.replace("ession", "") + take_dir.replace("ake", "") + '.csv'
        new_b = "csv_data/" + person_b + "_" + person_a + "_" + session_dir.replace("ession", "") + take_dir.replace("ake", "") + '.csv'
        f, f_len = convert_file(os.path.join(take_path, file_a), new_a)
        frames.append(f)
        frame_length.append(f_len)
        f, f_len
        convert_file(os.path.join(take_path, file_b), new_b)
        frames.append(f)
        frame_length.append(f_len)

106527 0.0111111111111
10846938 69.5919 17.3793
106527 0.0111111111111
10846938 69.5919 17.3793
76388 0.0111111111111
10846938 69.5919 17.3793
76388 0.0111111111111
10846938 69.5919 17.3793
6228 0.0111111111111
10846938 69.5919 17.3793
6228 0.0111111111111
10846938 69.5919 17.3793
55373 0.0111111111111
10846938 69.5919 17.3793
55373 0.0111111111111
10846938 69.5919 17.3793
63384 0.0111111111111
10846938 69.5919 17.3793
63384 0.0111111111111
10846938 69.5919 17.3793
21781 0.0111111111111
10846938 69.5919 17.3793
21781 0.0111111111111
10846938 69.5919 17.3793
57616 0.0111111111111
10846938 69.5919 17.3793
57616 0.0111111111111
10846938 69.5919 17.3793
32587 0.0111111111111
10846938 69.5919 17.3793
32587 0.0111111111111
10846938 69.5919 17.3793
62917 0.0111111111111
10846938 69.5919 17.3793
62917 0.0111111111111
10846938 69.5919 17.3793
61233 0.0111111111111
10846938 69.5919 17.3793
61233 0.0111111111111
10846938 69.5919 17.3793
37152 0.0111111111111
10846938 69.5919 17.3793
37152 0.01111

In [140]:
print(frame_length)

[106527]
